<a href="https://colab.research.google.com/github/Sathwikagoud06/Math-Agent/blob/main/Math_Agent__AI__Planet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain qdrant-client sentence-transformers sympy streamlit dspy tavily-python openai


In [ ]:
!pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()  # This will read the .env file and load the keys


False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import os
from dotenv import load_dotenv

# Load your .env file with the full path
load_dotenv("/content/drive/MyDrive/MathAgentProject/math_agent_env.env")

# Get environment variables
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")


In [ ]:
import os

folder_path = "/content/drive/MyDrive/MathAgentProject"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"✅ Folder created: {folder_path}")
else:
    print(f"ℹ️ Folder already exists: {folder_path}")


ℹ️ Folder already exists: /content/drive/MyDrive/MathAgentProject


In [28]:
import os
from dotenv import load_dotenv

# Load the .env file from your folder
load_dotenv("/content/drive/MyDrive/MathAgentProject/math_agent_env.env")

# Access the environment variables
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


**step 1(creating folder)**

In [ ]:
!pip install qdrant-client sentence-transformers --quiet

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from sentence_transformers import SentenceTransformer


In [ ]:
import os

# ✅ Step 1.1: Create safe project folder
project_dir = "/content/drive/MyDrive/MathAgentProject"
os.makedirs(project_dir, exist_ok=True)

# ✅ Step 1.2: Change directory to project folder
%cd {project_dir}

# ✅ Step 1.3: Install required packages
!pip install sympy tiktoken transformers sentence-transformers \
    qdrant-client faiss-cpu gradio openai langchain \
    tavily-python DSPy -q


/content/drive/MyDrive/MathAgentProject


step 2 knowledge_base.py

In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/knowledge_base.py
# knowledge_base.py

from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from dotenv import load_dotenv
import os
import logging
import numpy as np

logging.basicConfig(level=logging.INFO)

class KnowledgeBase:
    def __init__(self):
        load_dotenv("/content/drive/MyDrive/MathAgentProject/math_agent_env.env")

        self.QDRANT_URL = os.getenv("QDRANT_URL")
        self.QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

        if not self.QDRANT_URL or not self.QDRANT_API_KEY:
            raise ValueError("❌ Qdrant credentials not found. Please check your .env file and path.")

        self.collection_name = "math_agent_knowledge_base"
        self.vector_size = 768  # all-mpnet-base-v2
        self.distance = Distance.COSINE

        self.model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
        self.client = QdrantClient(
            url=self.QDRANT_URL,
            api_key=self.QDRANT_API_KEY,
            prefer_grpc=False
        )

        self._setup_collection()

    def _setup_collection(self):
        if not self.client.collection_exists(self.collection_name):
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(size=self.vector_size, distance=self.distance),
            )
            logging.info(f"✅ Created collection: {self.collection_name}")
        else:
            logging.info(f"✅ Collection already exists: {self.collection_name}")

    def upload_knowledge(self):
        data = [
            {"id": 1, "question": "integral of e**(-x**2)", "answer": "(sqrt(pi)/2)*erf(x) + C"},
            {"id": 2, "question": "derivative of sin(x)", "answer": "cos(x)"},
            {"id": 3, "question": "limit of 1/x as x approaches infinity", "answer": "0"},
        ]

        questions = [item["question"] for item in data]
        embeddings = self.model.encode(questions)

        points = []
        for item, vector in zip(data, embeddings):
            if isinstance(vector, np.ndarray):
                vector = vector.tolist()
            points.append({
                "id": item["id"],
                "vector": vector,
                "payload": {
                    "question": item["question"],
                    "answer": item["answer"]
                }
            })

        self.client.upsert(collection_name=self.collection_name, points=points)
        logging.info(f"✅ Uploaded {len(points)} entries to Qdrant")

    def search(self, query):
        query_vector = self.model.encode(query)
        if isinstance(query_vector, np.ndarray):
            query_vector = query_vector.tolist()

        results = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_vector,
            limit=1
        )

        if results and len(results) > 0:
            match = results[0]
            score = getattr(match, "score", 0)
            if score > 0.8:
                return match.payload["question"], match.payload["answer"], score

        return None, None, 0


# 🚀 Automatically upload knowledge when script is run directly
if __name__ == "__main__":
    try:
        kb = KnowledgeBase()
        kb.upload_knowledge()
        print("✅ Knowledge base uploaded successfully!")
    except Exception as e:
        print("❌ Error uploading knowledge base:", e)


Overwriting /content/drive/MyDrive/MathAgentProject/knowledge_base.py


In [ ]:
from knowledge_base import KnowledgeBase

kb = KnowledgeBase()
kb.upload_knowledge()
q, a, score = kb.search("what is the derivative of sin(x)?")
print("Top Match:", q)
print("Answer:", a)
print("Score:", score)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Top Match: derivative of sin(x)
Answer: cos(x)
Score: 0.91342044


step 3 web_search.py

In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/web_search.py
import os
import requests
from duckduckgo_search import DDGS
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("/content/drive/MyDrive/MathAgentProject/math_agent_env.env")

def search_web(query):
    """
    Attempts to retrieve a relevant answer for a math question from Tavily (primary)
    and falls back to DuckDuckGo if Tavily fails.
    """

    try:
        tavily_key = os.getenv("TAVILY_API_KEY")
        if not tavily_key:
            raise ValueError("TAVILY_API_KEY not found in environment variables. Please set it in your .env file.")

        response = requests.post(
            "https://api.tavily.com/search",
            headers={"Authorization": f"Bearer {tavily_key}"},
            json={"query": query, "search_depth": "basic"},
            timeout=10
        )
        if response.ok:
            data = response.json()
            if data.get("results"):
                return data["results"][0].get("content", "").strip()
            else:
                print("✅ Tavily responded but returned no results.")
        else:
            print(f"❌ Tavily error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"⚠️ Tavily failed: {e}")

    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=1))
            if results and "body" in results[0]:
                return results[0]["body"].strip()
            else:
                print("✅ DuckDuckGo responded but returned no body.")
    except Exception as e:
        print(f"⚠️ DuckDuckGo failed: {e}")

    return "Web search did not return relevant content."


Overwriting /content/drive/MyDrive/MathAgentProject/web_search.py


In [ ]:
import os
tavily_key = os.getenv("TAVILY_API_KEY")


step 4 math_solver.py

In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/math_solver.py
from sympy import Symbol, diff, integrate, limit, sympify, solve, simplify
from sympy.parsing.sympy_parser import parse_expr
from sympy.core.sympify import SympifyError
from sympy_fallback import sympy_fallback  # fallback method

def solve_math_question(question: str) -> str:
    """
    Tries solving the math question using SymPy. Falls back to web/LLM if needed.
    """
    try:
        question = question.lower()

        # Case 1: Derivative
        if "derivative" in question or "differentiate" in question:
            expr = question.split("of")[-1].strip()
            x = Symbol('x')
            result = diff(expr, x)
            return f"SymPy: The derivative is {result}"

        # Case 2: Integration
        elif "integrate" in question or "integration" in question:
            expr = question.split("of")[-1].strip()
            x = Symbol('x')
            result = integrate(expr, x)
            return f"SymPy: The integral is {result} + C"

        # Case 3: Limit
        elif "limit" in question and "as" in question:
            parts = question.split("limit of")[-1].strip().split(" as ")
            expr = parts[0].strip()
            var_part = parts[1].replace("approaches", "->").strip()
            var_name, value = [x.strip() for x in var_part.split("->")]
            x = Symbol(var_name)
            result = limit(sympify(expr), x, sympify(value))
            return f"SymPy: The limit is {result}"

        # Case 4: Solve
        elif "solve" in question:
            expr = question.split("solve")[-1].strip()
            x = Symbol('x')
            eq = parse_expr(expr)
            solution = solve(eq, x)
            return f"SymPy: Solution is {solution}"

        # Case 5: Simplify anything else
        else:
            expr = parse_expr(question)
            result = simplify(expr)
            return f"SymPy: Simplified result is {result}"

    except (SympifyError, Exception) as e:
        print("⚠️ SymPy failed, trying fallback...")
        try:
            fallback_result = sympy_fallback(question)
            return f"{fallback_result}"
        except Exception as inner_e:
            return f"SymPy Error: {str(inner_e)}"


Overwriting /content/drive/MyDrive/MathAgentProject/math_solver.py


step 5 feedback_loop.py

In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/feedback_loop.py
def get_human_feedback(question, agent_answer):
    """
    Handles Human-in-the-Loop Feedback.
    Prompts user to validate or refine the AI-generated answer.

    Parameters:
    - question (str): The original math question asked.
    - agent_answer (str): The answer generated by the system.

    Returns:
    - str: Final answer after optional human feedback.
    """

    print("\n🤖 Final Answer Before Feedback:", agent_answer)
    print("👩‍🎓 Student Feedback: Was this answer helpful?")
    print("✏️ Type 'yes' to accept, or write your corrected answer below:")

    feedback = input("👉 Feedback: ").strip().lower()

    if feedback == "yes":
        print("✅ Human confirmed the answer is correct.")
        return agent_answer

    elif feedback:
        print("✅ Feedback noted. Updating answer based on human input.")
        # Optional: store this feedback for future model improvement (bonus)
        return f"✅ Updated Based on Feedback: {feedback}"

    else:
        print("⚠️ No valid feedback provided. Keeping original answer.")
        return agent_answer


Overwriting /content/drive/MyDrive/MathAgentProject/feedback_loop.py


step 6 router_agent.py

In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/router_agent.py
from guardrails import input_guardrail, output_guardrail
from knowledge_base import KnowledgeBase
from web_search import search_web
from math_solver import solve_math_question
from feedback_loop import get_human_feedback
from sympy_fallback import sympy_fallback

class MathAgent:
    def __init__(self):
        self.kb = KnowledgeBase()

    def process_question(self, question: str) -> str:
        print("\n🤖 Received question:", question)

        # Step 0: Input Guardrail
        is_valid, message = input_guardrail(question)
        if not is_valid:
            return f"🚫 Input blocked: {message}"

        # Step 1: Search Knowledge Base
        kb_question, kb_answer, score = self.kb.search(question)
        if kb_answer and score > 0.8:
            print("✅ Answer found in Knowledge Base.")
            answer = f"📚 KB Answer:\n{kb_answer}\n(Score: {score:.2f})"
        else:
            print("🔎 Not found in KB. Moving to web search...")

            # Step 2: Web Search
            web_answer = search_web(question)
            print("🌐 Web Answer:", web_answer)

            # Step 3: Try solving using SymPy
            print("🧮 Trying SymPy...")
            sympy_result = solve_math_question(question)
            print("🧾 SymPy Output:", sympy_result)

            # Step 4: If SymPy fails, try fallback (e.g., LLM or heuristic)
            if not sympy_result or "SymPy Error" in sympy_result or "Not implemented" in sympy_result:
                print("⚠️ SymPy failed. Trying fallback...")
                try:
                    sympy_result = sympy_fallback(question)
                    print("✅ Fallback successful:", sympy_result)
                except Exception as e:
                    print("❌ Fallback failed:", str(e))
                    sympy_result = ""

            # Step 5: Decide which answer to prefer
            if sympy_result and "Error" not in sympy_result and "Not implemented" not in sympy_result:
                answer = f"🧮 Math Solver:\n{sympy_result}"
            elif web_answer:
                answer = f"🌐 Web Answer:\n{web_answer}"
            else:
                answer = "❌ Sorry, I couldn't solve this question."

        # Step 6: Output Guardrail
        final_safe_output = output_guardrail(answer)

        # Step 7: Human-in-the-loop Feedback
        reviewed_output = get_human_feedback(question, final_safe_output)

        return reviewed_output


# 🧪 For testing in Colab
if __name__ == "__main__":
    agent = MathAgent()
    while True:
        user_q = input("\n💬 Ask a math question (or type 'exit'): ")
        if user_q.lower() in ["exit", "quit"]:
            break
        response = agent.process_question(user_q)
        print("\n🟩 Final Answer:\n", response)


Overwriting /content/drive/MyDrive/MathAgentProject/router_agent.py


step 7 app.py

In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/app.py
from dotenv import load_dotenv
import os
import sys

# Load environment variables from your env file
load_dotenv("/content/drive/MyDrive/MathAgentProject/math_agent_env.env")

from router_agent import MathAgent

def run_cli_agent():
    print("\n📘 Welcome to the Human-in-the-Loop Math Agent (CLI)!")
    print("ℹ️  Type your math questions below.")
    print("🔚 Type 'exit' or 'quit' to end the session.\n")
    print("💡 Example questions: 'derivative of sin(x)', 'solve x**2 - 4 = 0', 'limit of 1/x as x->infinity'\n")

    agent = MathAgent()

    while True:
        try:
            user_question = input("📥 Enter a math question: ").strip()

            if user_question.lower() in ['exit', 'quit']:
                print("\n👋 Exiting Math Agent. Goodbye!")
                break

            if not user_question:
                print("⚠️ Please enter a valid question.")
                continue

            print("\n🧠 Thinking...\n")
            result = agent.process_question(user_question)
            print("\n🤖 Final Answer:", result)

        except KeyboardInterrupt:
            print("\n🛑 Interrupted. Exiting gracefully.")
            break

        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
            continue


def run_streamlit_app():
    import streamlit as st

    st.title("🧮 Human-in-the-Loop Math Agent (Web UI)")
    st.write("Ask me any math question, e.g., 'derivative of sin(x)', 'solve x**2 - 4 = 0', etc.")

    agent = MathAgent()

    question = st.text_input("Enter your math question:")

    if st.button("Solve"):
        if not question.strip():
            st.warning("Please enter a valid math question.")
        else:
            with st.spinner("Thinking..."):
                answer = agent.process_question(question)
            st.markdown(f"**Answer:** {answer}")



if __name__ == "__main__":
    if len(sys.argv) > 1 and sys.argv[1] == "web":
        # Run the Streamlit app if "web" arg is passed
        run_streamlit_app()
    else:
        # Otherwise run the CLI version by default
        run_cli_agent()


Overwriting /content/drive/MyDrive/MathAgentProject/app.py


step 8 guardrails.py

In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/guardrails.py
def input_guardrail(user_input):
    """
    Ensures only math-related questions are processed.
    Blocks inappropriate or unrelated content.
    Returns:
    - (bool, str): (True, "") if input is allowed, else (False, "error message")
    """
    blocked_keywords = ["politics", "violence", "game", "movie", "actor", "adult", "celebrity"]

    for word in blocked_keywords:
        if word.lower() in user_input.lower():
            return False, f"❌ Question blocked due to inappropriate keyword: '{word}'"

    if not user_input.strip():
        return False, "⚠️ Question cannot be empty."

    return True, ""  # ✅ Return tuple: allowed and empty message



def output_guardrail(response):
    """
    Ensures output is appropriate, educational, and non-harmful.
    Returns:
    - str: sanitized response or rejection message
    """
    if not response or len(response.strip()) == 0:
        return "⚠️ Output rejected: The system could not generate a safe answer."

    if any(bad_word in response.lower() for bad_word in ["kill", "bomb", "nude", "hack"]):
        return "🚫 Output rejected due to unsafe content."

    return response


Overwriting /content/drive/MyDrive/MathAgentProject/guardrails.py


In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/colab_run.py
from dotenv import load_dotenv
from router_agent import MathAgent

# Load environment variables (optional)
load_dotenv("/content/drive/MyDrive/MathAgentProject/math_agent_env.env")  # Update path if needed

# Create an instance of the Math Agent
agent = MathAgent()

# ✅ Provide your test question here (change as needed)
question = "limit of sin(x)/x as x approaches 0"

print("\n📥 Question:", question)
print("\n🧠 Thinking...\n")
answer = agent.process_question(question)
print("\n🤖 Final Answer:", answer)


Overwriting /content/drive/MyDrive/MathAgentProject/colab_run.py


In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/colab_test.py
from dotenv import load_dotenv
from router_agent import MathAgent

# Load environment variables (optional)
load_dotenv("/content/drive/MyDrive/MathAgentProject/math_agent_env.env")  # Update path if needed

agent = MathAgent()

questions = [
    "limit of sin(x)/x as x approaches 0",
    "derivative of x**3 + 2*x",
    "solve x**2 - 4 = 0",
    "integral of e**(-x**2)",
    "simplify (x**2 - 1)/(x - 1)"
]

print("\n📘 Running batch test for multiple math questions...\n")

for i, question in enumerate(questions, 1):
    print(f"📥 Question {i}: {question}")
    print("🧠 Thinking...\n")
    answer = agent.process_question(question)
    print(f"🤖 Answer {i}: {answer}\n")
    print("-" * 40)

print("✅ Batch test completed.")


Overwriting /content/drive/MyDrive/MathAgentProject/colab_test.py


In [ ]:
%%writefile /content/drive/MyDrive/MathAgentProject/sympy_fallback.py
from sympy import symbols, sympify, diff, integrate, limit, oo, solve, simplify
from sympy import log
from sympy.parsing.sympy_parser import (
    parse_expr, standard_transformations, implicit_multiplication_application
)
from sympy.core.sympify import SympifyError
import re  # Needed for text cleaning

# Define symbols globally
x, n = symbols('x n')

# Add implicit multiplication to transformations to handle inputs like 5x, 2sin(x), etc.
transformations = standard_transformations + (implicit_multiplication_application,)

def sympy_fallback(question: str) -> str:
    try:
        question = question.lower().strip()

        # Remove invisible/non-standard Unicode characters like ⁡, 𝑥, → etc.
        question = re.sub(r"[^\x00-\x7F]+", "", question)

        # Derivative handling
        if "derivative of" in question:
            expr_text = question.split("derivative of")[-1].strip().rstrip("?")

            # Fix missing parentheses in rational expressions (e.g. "x+1/x-1")
            if "/" in expr_text and "(" not in expr_text:
                parts = expr_text.split("/")
                if len(parts) == 2:
                    numerator = parts[0].strip()
                    denominator = parts[1].strip()
                    expr_text = f"({numerator})/({denominator})"

            expr = parse_expr(expr_text, transformations=transformations, evaluate=False)
            derivative = simplify(diff(expr, x))
            return f"SymPy: The derivative is {derivative}"

        # Integral handling
        elif "integral of" in question:
            expr_text = question.split("integral of")[-1].strip().rstrip("?")

            # Special case for Gaussian integral
            if expr_text == "e**(-x**2)":
                return "(sqrt(pi)/2)*erf(x) + C"

            expr = parse_expr(expr_text, transformations=transformations, evaluate=False)
            result = integrate(expr, x)
            return f"SymPy: The integral is {result} + C"

        # Limit handling
        elif "limit of" in question:
            if "as" in question and "approaches" in question:
                expr_text = question.split("limit of")[-1].split("as")[0].strip()
                var_part = question.split("as")[-1].strip().rstrip("?")
                var_name, _, approach_val = var_part.partition("approaches")
                var_name = var_name.strip()
                approach_val = approach_val.strip()

                var = symbols(var_name)
                expr = parse_expr(expr_text, transformations=transformations, evaluate=False)

                if approach_val == "infinity":
                    lim = limit(expr, var, oo)
                elif approach_val == "-infinity":
                    lim = limit(expr, var, -oo)
                else:
                    lim = limit(expr, var, sympify(approach_val))
                return f"SymPy: The limit is {lim}"

        # Equation solving
        elif "solve" in question or "=" in question:
            if "solve" in question:
                eq_text = question.split("solve")[-1].strip().rstrip("?")
            else:
                eq_text = question

            if "=" in eq_text:
                lhs_text, rhs_text = eq_text.split("=", 1)
                lhs = parse_expr(lhs_text.strip(), transformations=transformations, evaluate=False)
                rhs = parse_expr(rhs_text.strip(), transformations=transformations, evaluate=False)
                equation = lhs - rhs
            else:
                equation = parse_expr(eq_text, transformations=transformations, evaluate=False)

            solutions = solve(equation, x)

            if solutions:
                return f"SymPy: The solutions are {solutions}"
            else:
                return "SymPy: No solutions found."

        # Not recognized question type
        return "SymPy: Not implemented for this type."

    except SympifyError:
        return "SymPy: Could not parse the expression."
    except Exception as e:
        return f"SymPy Error: {str(e)}"


Overwriting /content/drive/MyDrive/MathAgentProject/sympy_fallback.py


In [ ]:
!pip install duckduckgo-search -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#(limit of 1/x as x -> infinity → (Should be in KB)

#derivative of sin(x) → (Should be in KB)

#integral of e**(-x**2) → (Should be in KB)

#solve x**2 - 5*x + 6 = 0 → (Should go to SymPy)

#limit of sin(x)/x as x->0 → (Test if it does SymPy/web fallback)
# Derivative of e^(x^2)


In [ ]:
!python3 /content/drive/MyDrive/MathAgentProject/app.py

2025-05-16 13:39:43.787136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747402783.826825   21520 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747402783.835983   21520 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

📘 Welcome to the Human-in-the-Loop Math Agent (CLI)!
ℹ️  Type your math questions below.
🔚 Type 'exit' or 'quit' to end the session.

💡 Example questions: 'derivative of sin(x)', 'solve x**2 - 4 = 0', 'limit of 1/x as x->infinity'

📥 Enter a math question: derivative of x**2 + 3*x + 2

🧠 Thinking...


🤖 Received question: derivative of x**2 + 3*x + 2
🔎 Not found in KB. Moving to web search...
🌐 Web Answer: Derivative Calculator • Wi

!streamlit run /content/drive/MyDrive/MathAgentProject/app.py -- web


In [ ]:
!pip install streamlit pyngrok


In [ ]:
import os
from pyngrok import ngrok

# Start Streamlit app in background
os.system("streamlit run /content/drive/MyDrive/MathAgentProject/app.py &")


0

In [ ]:
# Install pyngrok if not installed
!pip install pyngrok --quiet

from pyngrok import ngrok
import os
import time

# Your ngrok authtoken (replace with your actual token)
NGROK_AUTH_TOKEN = "2xBGlJiSu83s4XWirNiOWR2MJLM_3uSyrZKGVqrGoytVRWrZg"

# Authenticate ngrok with your authtoken
!ngrok authtoken {NGROK_AUTH_TOKEN}

# Kill any existing ngrok tunnels (optional cleanup)
!killall ngrok > /dev/null 2>&1

# Start your Streamlit app in the background
# Make sure your app.py path is correct
os.system("streamlit run /content/drive/MyDrive/MathAgentProject/app.py &")

time.sleep(5)  # Wait a few seconds for Streamlit app to start

# Open a public tunnel on the default Streamlit port 8501
public_url = ngrok.connect(8501).public_url

print(f"🚀 Your Streamlit app is live at: {public_url}")
print("Open this URL in a new browser tab to access your app.")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Your Streamlit app is live at: https://0d9f-35-245-175-37.ngrok-free.app
Open this URL in a new browser tab to access your app.
